
# Kubernetes Lab

This lab is presented in several sections

- [Setup](0.SETUP.ipynb)
- [Concepts](1.Concepts.ipynb)
- [Kubectl Basics](2.kubectl_basics.ipynb)
- [Running pods, deployments](3.pods.ipynb)
- [Exposing Services](4.services.ipynb)
- [Maintenance Operations](5.maintenance.ipynb)
- [Tools](6.tools.ipynb)
- [Resources](9.Resources.ipynb)

## Contributions

Your contributions to this tutorial are welcome, either via

- Opening issues on github at [ContainerOrchestration/Labs](https://github.com/ContainerOrchestration/Labs)
- Submitting PRs
- tweets to @mjbright
